# 🚞 Zero-shot RE Training

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# # if you're running this in a colab notebook, you can run this cell to install the necessary dependencies
# !git clone https://github.com/jackboyla/GLiREL.git
# !cd GLiREL && pip install -e .
# !python -m spacy download en_core_web_sm

In [6]:
from glirel import GLiREL

save_path = 'logs/zero_rel/zero_rel-2024-06-02__07-59-38/model_39000'
model = GLiREL.from_pretrained(save_path)
# model

config.json not found in /home/jackboylan/GLiREL/logs/zero_rel/zero_rel-2024-06-02__07-59-38/model_39000
2024-06-03 16:13:12,801 - huggingface_hub.hub_mixin - WARNING - config.json not found in /home/jackboylan/GLiREL/logs/zero_rel/zero_rel-2024-06-02__07-59-38/model_39000


# Inference

To infer, the model needs `tokens`, `NER`, and `zero shot labels`.

### Eval data

In [7]:
import json
with open('./data/few_rel_all.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

i = 0

tokens = data[i]['tokenized_text']
ner = data[i]['ner']
labels = list(set([r['relation_text'] for r in data[i]['relations']]))
print(tokens)
print()
print(ner)
print(labels)

['Derren', 'Nesbitt', 'had', 'a', 'history', 'of', 'being', 'cast', 'in', '"', 'Doctor', 'Who', '"', ',', 'having', 'played', 'villainous', 'warlord', 'Tegana', 'in', 'the', '1964', 'First', 'Doctor', 'serial', '"', 'Marco', 'Polo', '"', '.']

[[26, 28, 'Q2989881', 'Marco Polo'], [22, 24, 'Q2989412', 'First Doctor']]
['characters']


In [8]:
labels = ['country of origin', 'licensed to broadcast to', 'father', 'followed by'] + labels
print(labels)

['country of origin', 'licensed to broadcast to', 'father', 'followed by', 'characters']


In [9]:
relations = model.predict_relations(tokens, labels, threshold=0.01, ner=ner)

print('Number of relations:', len(relations))  # num entity pairs (both directions) * num classes.... provided they're over the threshold

sorted_data_desc = sorted(relations, key=lambda x: x['score'], reverse=True)
print("\nDescending Order by Score:")
for item in sorted_data_desc:
    print(item)

Number of relations: 6

Descending Order by Score:
{'head_pos': [22, 24], 'tail_pos': [26, 28], 'head_text': ['First', 'Doctor'], 'tail_text': ['Marco', 'Polo'], 'label': 'characters', 'score': 0.06290033459663391}
{'head_pos': [26, 28], 'tail_pos': [22, 24], 'head_text': ['Marco', 'Polo'], 'tail_text': ['First', 'Doctor'], 'label': 'characters', 'score': 0.06271035224199295}
{'head_pos': [26, 28], 'tail_pos': [22, 24], 'head_text': ['Marco', 'Polo'], 'tail_text': ['First', 'Doctor'], 'label': 'followed by', 'score': 0.017688073217868805}
{'head_pos': [22, 24], 'tail_pos': [26, 28], 'head_text': ['First', 'Doctor'], 'tail_text': ['Marco', 'Polo'], 'label': 'followed by', 'score': 0.01758633926510811}
{'head_pos': [26, 28], 'tail_pos': [22, 24], 'head_text': ['Marco', 'Polo'], 'tail_text': ['First', 'Doctor'], 'label': 'father', 'score': 0.012392115779221058}
{'head_pos': [22, 24], 'tail_pos': [26, 28], 'head_text': ['First', 'Doctor'], 'tail_text': ['Marco', 'Polo'], 'label': 'father',

### Real-world example

In [13]:
# Real-world example
import spacy
nlp = spacy.load('en_core_web_sm')

text = "Apple Inc. was founded by Steve Jobs, Steve Wozniak, and Ronald Wayne in April 1976. The company is headquartered in Cupertino, California."

# text = "Jack Dorsey's father, Tim Dorsey, is a licensed pilot. Jack met his wife Sarah Paulson in New York in 2003. They have one son, Edward."

labels = [
    'co-founder', 
    'country of origin', 
    'licensed to broadcast to', 
    'no relation', 
    'parent', 
    'followed by', 
    'located in or next to body of water', 
    'spouse', 
    'child', 
    'founder', 
    'headquartered in', 
    'acquired by', 
    'subsidiary of'
    ]



def predict_and_show(text, labels):
    doc = nlp(text)
    print(f"Text: {text}")

    tokens = [token.text for token in doc]

    # NOTE: the end index should be inclusive
    ner = [[ent.start, (ent.end - 1), ent.label_, ent.text] for ent in doc.ents]
    print(f"Entities detected: {ner}")

    relations = model.predict_relations(tokens, labels, threshold=0.0, ner=ner, top_k=1)

    print('Number of relations:', len(relations))

    sorted_data_desc = sorted(relations, key=lambda x: x['score'], reverse=True)
    print("\nDescending Order by Score:")
    for item in sorted_data_desc:
        print(f"{item['head_text']} --> {item['label']} --> {item['tail_text']} | score: {item['score']}")

predict_and_show(text, labels)

Text: Apple Inc. was founded by Steve Jobs, Steve Wozniak, and Ronald Wayne in April 1976. The company is headquartered in Cupertino, California.
Entities detected: [[0, 2, 'ORG', 'Apple Inc.'], [5, 7, 'PERSON', 'Steve Jobs'], [8, 10, 'PERSON', 'Steve Wozniak'], [12, 14, 'PERSON', 'Ronald Wayne'], [15, 17, 'DATE', 'April 1976'], [23, 24, 'GPE', 'Cupertino'], [25, 26, 'GPE', 'California']]
Number of relations: 42

Descending Order by Score:
['Steve', 'Jobs'] --> headquartered in --> ['Cupertino'] | score: 0.9002836346626282
['Steve', 'Jobs'] --> headquartered in --> ['California'] | score: 0.8964382410049438
['Steve', 'Wozniak'] --> headquartered in --> ['Cupertino'] | score: 0.8919743299484253
['Steve', 'Wozniak'] --> headquartered in --> ['California'] | score: 0.888839840888977
['Apple', 'Inc.'] --> headquartered in --> ['Cupertino'] | score: 0.8876587748527527
['Apple', 'Inc.'] --> headquartered in --> ['California'] | score: 0.8779582381248474
['Cupertino'] --> headquartered in -->

In [11]:
text = "Jack knows Gill. They live in the same house in London. They are not related."
labels = ['family relation', 'knows', 'lives with', 'loves', 'licensed to broadcast to', 'father', 'followed by', 'no relation', 'lives in',]
predict_and_show(text, labels)

Text: Jack knows Gill. They live in the same house in London. They are not related.
Entities detected: [[0, 1, 'PERSON', 'Jack'], [2, 3, 'PERSON', 'Gill'], [11, 12, 'GPE', 'London']]
Number of relations: 6

Descending Order by Score:
['Jack'] --> lives in --> ['London'] | score: 0.8900772929191589
['Gill'] --> lives in --> ['London'] | score: 0.8764218688011169
['London'] --> lives in --> ['Jack'] | score: 0.7262811064720154
['London'] --> lives in --> ['Gill'] | score: 0.663715660572052
['Jack'] --> knows --> ['Gill'] | score: 0.5179734826087952
['Gill'] --> knows --> ['Jack'] | score: 0.5167464017868042


In [12]:
import spacy
from spacy.tokens import Span
from spacy import displacy

def visualize_relation(text, relations):
    nlp = spacy.blank("en")
    doc = nlp(text)

    # Manually set dependency relations to visualize relations
    for token in doc:
        token.dep_ = "dep"  # default to 'dep'

    spans = []
    # Calculate character offsets for each entity
    for rel in relations:
        head = Span(doc, rel['head_pos'][0], rel['head_pos'][1], label=rel['head_text'])
        tail = Span(doc, rel['tail_pos'][0], rel['tail_pos'][1], label=rel['tail_text'])

        doc.ents += (head, tail)

        # Mock dependencies
        head_root = head.root
        tail_root = tail.root

        head_root.dep_ = "rel"  # Relation type can be customized
        head_root.head = tail_root  # Point head to tail

    options = {"fine_grained": True}
    displacy.render(doc, style="dep", options=options, jupyter=True)

# Example data
text = "Binsey located in or next to body of water River Thames"
relations = [
    {'head_pos': [0, 1], 'tail_pos': [9, 11], 'head_text': 'Binsey', 'tail_text': 'River Thames', 'label': 'located in or next to body of water', 'score': 0.9235768914222717},
    # {'head_pos': [9, 11], 'tail_pos': [0, 1], 'head_text': 'River Thames', 'tail_text': 'Binsey', 'label': 'located in or next to body of water', 'score': 0.12615662813186646}
]

visualize_relation(text, relations)
